In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import f1_score, accuracy_score, classification_report

import torch
from torch.utils.data import DataLoader

import json
import re

from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import DataCollatorWithPadding

from tqdm.notebook import tqdm
from functools import reduce

### Load data

In [2]:
mentions_df = pd.read_pickle('../data/mentions texts.pickle')
mentions_df.head(3)

,ChannelID,messageid,issuerid,MessageID,DateAdded,DatePosted,MessageText,IsForward
0,1197210433,5408,90,5408,2021-02-06 01:42:42,2020-04-29 07:29:01,?? Фокус недели #ФН Сегодня ????? ММК опублик...,False
1,1203560567,64803,57,64803,2021-02-06 01:47:00,2020-01-21 12:51:42,??#LSRG ЛСР - операционные результаты (2019г)...,False
2,1197210433,23389,152,23389,2021-07-21 13:46:31,2021-07-21 11:15:46,#CHMF Северсталь (CHMF) впервые поставила в Бр...,False


In [3]:
mentions_fact_df = pd.read_csv('../data/mentions.csv', index_col=0)
mentions_fact_df.head(3)

,ChannelID,messageid,issuerid
0,1197210433,5408,90
1,1203560567,64803,57
2,1197210433,23389,152


In [4]:
issuers_df = pd.read_excel('../data/issuers.xlsx', index_col=0)
issuers_df.head(3)

,issuerid,EMITENT_FULL_NAME,datetrackstart,datetrackend,BGTicker,OtherTicker
0,1,"""Акционерный коммерческий банк ""Держава"" публи...",2021-06-02 12:47:55.100,NaN,NaN,NaN
1,2,"""МОСКОВСКИЙ КРЕДИТНЫЙ БАНК"" (публичное акционе...",2021-06-02 12:47:55.100,NaN,CBOM RX,NaN
2,3,"""Российский акционерный коммерческий дорожный ...",2021-06-02 12:47:55.100,NaN,NaN,NaN


In [5]:
# Загружаем словарь со сгенерированными названиями компаний во всех возможных падежах
with open('../data/declensions.json', mode='r', encoding='utf-8') as f:
    declensions_dict = json.load(f)

### Load model

In [6]:
# model_name = 'Babelscape/wikineural-multilingual-ner'
model_name = 'viktoroo/sberbank-rubert-base-collection3'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

### Prepare dataset

In [7]:
# удаляем плохие данные и группируем по сообщениям
mentions_df = mentions_df[~mentions_df['issuerid'].isin([253, -2, -3])].copy()
train_df = mentions_df.groupby(['ChannelID', 'messageid', 'MessageText'], as_index=False)['issuerid'].apply(list)

Была попытка убрать все символы, кроме букв русского и английского алфавитов, некоторых знаков пунктуации и цифр,\
однако качество модели от этого стало только хуже

In [8]:
# symb_vocab = {}
# for txt in tqdm(train_df['MessageText'], total=train_df.shape[0]):
#     for symb in txt:
#         if symb in symb_vocab:
#             symb_vocab[symb] += 1
#         else:
#             symb_vocab[symb] = 1

In [9]:
# bad_symbols = [k for k, v in symb_vocab.items() if not re.compile(r'[a-zа-яё1-9,-.!? ]').match(k.lower())]

In [10]:
# for bs in tqdm(bad_symbols):
#     train_df['MessageText'] = train_df['MessageText'].str.replace(bs, '')
train_df['MessageText'] = train_df['MessageText'].str.replace('-ао', ' ао')
train_df['MessageText'] = train_df['MessageText'].str.replace('  ', ' ', n=50)
train_df['MessageText'] = train_df['MessageText'].str.replace(',,', ',', n=50)
train_df['MessageText'] = train_df['MessageText'].str.replace('..', '.', n=50)

Подготовка датасета в удобном для нейросети формате

In [11]:
train_set = Dataset.from_pandas(train_df)

In [12]:
train_set

Dataset({
    features: ['ChannelID', 'messageid', 'MessageText', 'issuerid'],
    num_rows: 16688
})

In [13]:
# токенизация текстов
mapped_set = (
    train_set
    .map(lambda x: tokenizer(x['MessageText'], padding=True, truncation=True, max_length=512),
         remove_columns=['ChannelID', 'messageid', 'MessageText', 'issuerid'])
)
mapped_set.set_format('torch')

Map:   0%|          | 0/16688 [00:00<?, ? examples/s]

In [14]:
data_collator = DataCollatorWithPadding(tokenizer, padding=True)

In [15]:
batch_size = 128

train_dataloader = DataLoader(
    mapped_set, 
    batch_size=batch_size, drop_last=False, num_workers=0, shuffle=False, collate_fn=data_collator
)

### Make predictions

In [17]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [18]:
model.to(device)

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(120138, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, e

Т.к. использумая модель предназначена на классификации каждого токена, то необходимо собрать выявленную сущность из соответствующих ей токенов\
Магические цифры: 3 - B-ORG (начальный токен сущноси) 4 - I-ORG (внутренний токен сущности)\
Как показала практика модель иногда выдает баги и начальным токеном бывает не только 3, поэтому алгоритм работает на основе цепочек из токенов 4

Поскольку требуется, чтобы модель работала быстро, обработка каждого тензора в каждом батче в цикле выглядит дорого. В связи с этим большинство кода реализовано на torch, за исключением моментов, где избежать цикла впринципе невозможно

In [19]:
preds = []

model.eval()
for batch in tqdm(train_dataloader):
    batch = batch.to(device)
    with torch.no_grad():
        model_outputs = model(
            input_ids=batch.input_ids,
            token_type_ids=batch.token_type_ids,
            attention_mask=batch.attention_mask
        )

    out_labels = torch.softmax(model_outputs.logits, dim=2).argmax(axis=2)
    out_labels = out_labels * batch.attention_mask # убираем PAD токены
    
    for i in range(batch.input_ids.shape[0]):
        tmp = out_labels[i]

        # ищем индексы цепочек вида: [3, 4, 4, 4] или [x, 4, 4, 4], где x - любой возможный токен
        tmp_roll = torch.roll(tmp, 1)
        tmp_roll_r = torch.roll(tmp, -1)
        begins = (((tmp == 4) & (tmp_roll != 4)).nonzero() - 1).squeeze(dim=1).cpu().detach().tolist()
        ends = (((tmp == 4) & (tmp_roll_r != 4)).nonzero() + 1).squeeze(dim=1).cpu().detach().tolist()

        # ищем индексы одиночныъ токенов 3
        single_3 = ((tmp == 3) & (tmp_roll_r != 4)).nonzero().squeeze(dim=1).cpu().detach()
        begins.extend(single_3.tolist())
        ends.extend((single_3 + 1).tolist())

        assert len(begins) == len(ends), i
        segments_ids = list(zip(begins, ends))

        # добавляем декодированные токены для каждого объекта выборки (т. е. для каждого сообщения)
        preds.append([tokenizer.decode(batch.input_ids[i][b:e]) for b, e in segments_ids])

  0%|          | 0/131 [00:00<?, ?it/s]

### Predictions processing

Для расчёта метрик нужно привести предсказания к нужному формату

In [22]:
preds_df = train_df[['ChannelID', 'messageid', 'issuerid']].copy()
preds_df['preds'] = preds
preds_df.head(3)

,ChannelID,messageid,issuerid,preds
0,1001029560,1113,[32],"[ростелекома, алле]"
1,1001029560,1177,[62],[goo. gl / wp9q5k]
2,1001029560,1501,[26],"[афк система, goo. gl / hrq1mx]"


In [23]:
# функция для поиска предсказанной сущности в словаре всевозможных падежей названий компаний
def find_id(names):
    res = []
    for name in names:
        if len(name) < 3:
            continue
        name = name.lower()
        pred_id = [int(k) for k, v in declensions_dict.items() if name in v['EMITENT_FULL_NAME'] or name in v['declensions']]
        res.extend(pred_id)
    return np.unique(res)

In [25]:
%%time
preds_df['preds'] = preds_df['preds'].apply(lambda l: list(map(lambda x: x.lower(), l)))
preds_df['preds_id'] = preds_df['preds'].apply(find_id)
preds_df.head(3)

CPU times: total: 6.48 s
Wall time: 6.48 s


,ChannelID,messageid,issuerid,preds,preds_id
0,1001029560,1113,[32],"[ростелекома, алле]",[142]
1,1001029560,1177,[62],[goo. gl / wp9q5k],[]
2,1001029560,1501,[26],"[афк система, goo. gl / hrq1mx]",[26]


In [26]:
# словари для кодирования и расходирования id предсказаний в промежутке [0..255)
label2id = {iid: i for i, iid in enumerate(issuers_df['issuerid'])}
id2label = {v: k for k, v in label2id.items()}

In [29]:
# бинаризация списка с id
# например [1, 3] --> [0, 1, 0, 1, 0, ..., 0]
def binarize_id_labels(labels):
    labels = list(map(lambda x: label2id[x], labels))
    return [0 if l not in labels else 1 for l in range(len(label2id))]

Далее приведём факты и предсказания к нужному формату и соединим вместе

In [30]:
mentions_fact_df = mentions_fact_df[~mentions_fact_df['issuerid'].isin([253, -2, -3])]
facts_df = mentions_fact_df.groupby(['ChannelID', 'messageid'], as_index=False).agg(list)
facts_df['facts_bin_id'] = facts_df['issuerid'].apply(binarize_id_labels)

In [31]:
compare_df = pd.merge(
    facts_df, preds_df[['ChannelID', 'messageid', 'preds_id']],
    on=['ChannelID', 'messageid'], how='left',
    validate='1:1'
).fillna('')

In [32]:
compare_df['preds_id'] = compare_df['preds_id'].apply(list)
compare_df['preds_bin_id'] = compare_df['preds_id'].apply(binarize_id_labels)

### Score predictions

Результаты получились довольно скудные, однако в случае дообучения на задачу NER на размеченном датасете - идея имеет место быть

In [33]:
facts = np.array(compare_df['facts_bin_id'].tolist())
preds = np.array(compare_df['preds_bin_id'].tolist())

In [34]:
f1_score(facts, preds, average='macro')

0.153067459823616

In [35]:
accuracy_score(facts, preds)

0.28286472148541114

In [36]:
print(classification_report(facts, preds))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.25      0.46      0.33       110
           2       0.00      0.00      0.00         0
           3       0.46      0.43      0.45       520
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.64      0.72      0.67      1264
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0
          10       0.48      0.47      0.47       504
          11       0.38      0.19      0.25       102
          12       0.00      0.00      0.00         0
          13       0.00      0.00      0.00         0
          14       0.00      0.00      0.00         0
          15       0.00      0.00      0.00         0
          16       0.00      0.00      0.00         3
          17       0.00    

C:\Users\lexabr\anaconda3\envs\gag_venv\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\lexabr\anaconda3\envs\gag_venv\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [37]:
# если сравнить только на вхождениях id, которые есть в изначальной разметке,
# то можно видеть, что многие классы (компании) предсказываются с точностью 1
# это показывает, что в целом модель выдает осмысленные результаты,
# однако добавляет к ним сущности, которые не были размечены

preds_2 = np.where(facts == 1, preds, 0)
print(classification_report(facts, preds_2))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.46      0.63       110
           2       0.00      0.00      0.00         0
           3       1.00      0.43      0.60       520
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       1.00      0.72      0.84      1264
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0
          10       1.00      0.47      0.64       504
          11       1.00      0.19      0.31       102
          12       0.00      0.00      0.00         0
          13       0.00      0.00      0.00         0
          14       0.00      0.00      0.00         0
          15       0.00      0.00      0.00         0
          16       0.00      0.00      0.00         3
          17       0.00    

C:\Users\lexabr\anaconda3\envs\gag_venv\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\lexabr\anaconda3\envs\gag_venv\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\lexabr\anaconda3\envs\gag_venv\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resul